In [1]:
cd ../

/Users/linafaik/Documents/projects/time-series-forecasting-models


In [2]:
import pandas as pd
import numpy as np
import os

from config import *
from src.data_processing import *
from src.metrics import *
from src.training import *

%load_ext autoreload
%autoreload 2


/Users/linafaik/Documents/projects/time-series-forecasting-models/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv("data/sales_train_evaluation.csv")
calendar_df = pd.read_csv("data/calendar.csv")
sell_prices_df = pd.read_csv("data/sell_prices.csv")

In [4]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [5]:
calendar_df["date"] = pd.to_datetime(calendar_df["date"], format="%Y-%m-%d")
calendar_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [6]:
sell_prices_df.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [8]:
long_df = reshape_sales_data(
    df=df, 
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    calendar_df=calendar_df,
    )

print(f"{len(long_df)} in long_df")
long_df.head()

59181090 in long_df


,id,item_id,dept_id,cat_id,store_id,state_id,date,sales
1612,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2011-01-29,3
32102,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2011-01-30,0
62592,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2011-01-31,0
93082,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2011-02-01,1
123572,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2011-02-02,4


In [ ]:
sales_df = (
    long_df.groupby(['date', 'item_id', 'store_id', 'state_id'],as_index=False)
    .agg({'sales': 'sum'})
    )

sales_df.head()

,date,store_id,state_id,sales
0,2011-01-29,CA_1,CA,4337
1,2011-01-29,CA_2,CA,3494
2,2011-01-29,CA_3,CA,4739
3,2011-01-29,CA_4,CA,1625
4,2011-01-29,TX_1,TX,2556


In [39]:
sales_enr_df = (
    long_df.groupby(['date', 'item_id', 'store_id', 'state_id'],as_index=False)
    .agg(sold_quantity=('sales','sum'))
    .merge(
        calendar_df[["date", "wm_yr_wk"]], 
        on="date", 
        how="left")
    .merge(
        sell_prices_df,
        on=["store_id", "item_id", "wm_yr_wk"],
        how="left"
    )
)

sales_enr_df["sold_amount"] = sales_enr_df["sold_quantity"]*sales_enr_df["sell_price"]

sales_enr_df.head()

,date,item_id,store_id,state_id,sold_quantity,wm_yr_wk,sell_price,sold_amount
0,2011-01-29,FOODS_1_001,CA_1,CA,3,11101,2.0,6.0
1,2011-01-29,FOODS_1_001,CA_2,CA,2,11101,2.0,4.0
2,2011-01-29,FOODS_1_001,CA_3,CA,1,11101,2.0,2.0
3,2011-01-29,FOODS_1_001,CA_4,CA,0,11101,2.0,0.0
4,2011-01-29,FOODS_1_001,TX_1,TX,0,11101,2.0,0.0


In [40]:
sales_enr_agg_df = (
    sales_enr_df
    .groupby(['date', 'store_id', 'state_id'],as_index=False)
    .agg({'sold_quantity': 'sum', 'sold_amount': 'sum'})
    .merge(
        calendar_df[["date", "event_type_1", "event_type_2"]], 
        on="date", 
        how="left"
        )
    )

sales_enr_agg_df.head()

,date,store_id,state_id,sold_quantity,sold_amount,event_type_1,event_type_2
0,2011-01-29,CA_1,CA,4337,10933.16,NaN,NaN
1,2011-01-29,CA_2,CA,3494,9101.52,NaN,NaN
2,2011-01-29,CA_3,CA,4739,11679.83,NaN,NaN
3,2011-01-29,CA_4,CA,1625,4561.59,NaN,NaN
4,2011-01-29,TX_1,TX,2556,6586.68,NaN,NaN


In [41]:
sales_enr_agg_df.date.min(), sales_enr_agg_df.date.max()

(Timestamp('2011-01-29 00:00:00'), Timestamp('2016-05-22 00:00:00'))

In [42]:
all_event_types = pd.Series(
    pd.concat([sales_enr_agg_df["event_type_1"], sales_enr_agg_df["event_type_2"]])
).dropna().unique()

for event in all_event_types:
    sales_enr_agg_df[f"event_{event.lower().replace(' ', '_')}"] = (
        (sales_enr_agg_df["event_type_1"] == event) |
        (sales_enr_agg_df["event_type_2"] == event)
    ).astype(int)
    
sales_enr_agg_df.head()

,date,store_id,state_id,sold_quantity,sold_amount,event_type_1,event_type_2,event_sporting,event_cultural,event_national,event_religious
0,2011-01-29,CA_1,CA,4337,10933.16,NaN,NaN,0,0,0,0
1,2011-01-29,CA_2,CA,3494,9101.52,NaN,NaN,0,0,0,0
2,2011-01-29,CA_3,CA,4739,11679.83,NaN,NaN,0,0,0,0
3,2011-01-29,CA_4,CA,1625,4561.59,NaN,NaN,0,0,0,0
4,2011-01-29,TX_1,TX,2556,6586.68,NaN,NaN,0,0,0,0


In [43]:
sales_enr_agg_df.to_csv("output/data_processed.gz", index=False)

In [4]:
sales_enr_agg_df.groupby("store_id",as_index=False).size()

,store_id,size
0,CA_1,1941
1,CA_2,1941
2,CA_3,1941
3,CA_4,1941
4,TX_1,1941
5,TX_2,1941
6,TX_3,1941
7,WI_1,1941
8,WI_2,1941
9,WI_3,1941


In [44]:
import plotly.express as px

px.line(
    sales_enr_agg_df.groupby(["date"],as_index=False).agg({"sold_quantity": "sum"}),
    x="date",
    y="sold_quantity",
    template="plotly_white",
    title="Total sales",
)

In [49]:
random_store_id = np.random.choice(sales_enr_agg_df['store_id'].unique())
ts_df = sales_enr_agg_df[sales_enr_agg_df['store_id'] == random_store_id].reset_index(drop=True)

px.line(
    ts_df,
    x="date",
    y="sold_quantity",
    template="plotly_white",
    title=f"Total sales for {random_store_id}",
)